In [1]:
import subprocess
from pydub import AudioSegment
import math

def extract_audio_from_video(video_path, autio_path):
    command = ["ffmpeg", "-i", video_path, "-vn", autio_path]

    subprocess.run(command)

def cut_audio_in_chunk(audio_path, chunk_size, chunk_folder):
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = chunk_size * 60 * 1000
    chunks = math.ceil(len(track) / chunk_len)

    for i in range(chunks):
        start_time = i * chunk_len
        end_time = (i + 1) * chunk_len
        chunk = track[start_time:end_time]
        chunk.export(f"{chunk_folder}/chunk_{i}.mp3", format="mp3")


#cut_audio_in_chunk("./files/podcast.mp3", 10, "./files/chunks")
# extract_audio_from_video("./files/podcast.mp4", "./files/podcast.mp3",) 영상파일을 음성파일로
# track.duration_seconds




In [5]:
import openai
import glob

# transcript = openai.Audio.transcribe("whisper-1", open("./files/chunks/chunk_0.mp3", "rb"), )
# transcript["text"]

def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(f"{chunk_folder}/*.mp3")
    final_transcript = ""
    for file in files:
        with open(file, "rb") as audio_file:
            transcript = openai.Audio.transcribe(
                "whisper-1", 
                audio_file, 
            )
            final_transcript += transcript["text"]
    with open(destination, "w") as file:
        file.write(final_transcript)
    

transcribe_chunks(
    "./files/chunks", 
    "./files/transcript.txt",
)


# transcript = openai.audio.transcriptions.create(
# model="whisper-1",
# file=open("./files/chunks/chunk_0.mp3", "rb"),
# language="en"
# )

